In [ ]:
pip install openai==0.28
pip install streamlit streamlit-webrtc librosa scikit-learn xgboost speechrecognition pyngrok
from pyngrok import ngrok

# Replace with ngrok token
ngrok.set_auth_token("key")

! pip install streamlit -q
!wget -q -O - ipv4.icanhazip.com
! streamlit run app.py & npx localtunnel --port 8501


App.py

In [ ]:
import streamlit as st
import librosa
import numpy as np
import joblib
import tensorflow as tf
import openai

# Load models
xgb_model = joblib.load('/content/drive/MyDrive/xgb_emotion_model.pkl')
cnn_model = tf.keras.models.load_model('/content/drive/MyDrive/cnn_emotion_model.h5')
label_encoder = joblib.load('/content/drive/MyDrive/label_encoder.pkl')

# OpenAI API key
openai.api_key ="api_key"

# Session state init
if "conversation_history" not in st.session_state:
    st.session_state.conversation_history = []
if "messages" not in st.session_state:
    st.session_state.messages = []
if "current_emotion" not in st.session_state:
    st.session_state.current_emotion = None
if "audio_file_name" not in st.session_state:
    st.session_state.audio_file_name = None
if "chat_ready" not in st.session_state:
    st.session_state.chat_ready = False

# Feature extraction
def extract_features_flat(file_path, max_pad_len=174):
    audio, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant') if pad_width > 0 else mfccs[:, :max_pad_len]
    return mfccs.flatten().reshape(1, -1)

def extract_features_2d(file_path, max_pad_len=174):
    audio, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant') if pad_width > 0 else mfccs[:, :max_pad_len]
    return mfccs[..., np.newaxis].reshape(1, 40, max_pad_len, 1)

# Sidebar
st.sidebar.title("🗂️ Conversations")
for i, convo in enumerate(st.session_state.conversation_history):
    label = f"{i + 1}. {convo['audio_file']} ({convo['emotion']})"
    if st.sidebar.button(label, key=f"sidebar_{i}"):
        st.session_state.messages = convo["chat"]
        st.session_state.current_emotion = convo["emotion"]
        st.session_state.audio_file_name = convo["audio_file"]
        st.session_state.chat_ready = True

if st.sidebar.button("➕ New Chat"):
    st.session_state.clear()
    st.session_state.conversation_history = []
    st.session_state.messages = []
    st.session_state.current_emotion = None
    st.session_state.audio_file_name = None
    st.session_state.chat_ready = False
    st.rerun()

# Title
st.title("🎙️ Emotion-Aware ChatGPT")

# Step 1: Upload and detect emotion
if not st.session_state.chat_ready:
    st.subheader("Step 1: Upload Audio to Detect Emotion")
    model_choice = st.radio("Select Model", ["XGBoost", "CNN"])
    uploaded_file = st.file_uploader("Upload a .wav file", type=["wav"])

    if uploaded_file:
        with open("temp.wav", "wb") as f:
            f.write(uploaded_file.read())

        features = extract_features_flat("temp.wav") if model_choice == "XGBoost" else extract_features_2d("temp.wav")
        pred = xgb_model.predict(features)[0] if model_choice == "XGBoost" else np.argmax(cnn_model.predict(features), axis=1)[0]
        emotion = label_encoder.inverse_transform([pred])[0]

        st.session_state.current_emotion = emotion
        st.session_state.audio_file_name = uploaded_file.name
        st.session_state.messages = [
            {
                "role": "system",
                "content": f"You are a helpful assistant. The user is feeling {emotion}. Be supportive and empathetic."
            }
        ]
        st.session_state.chat_ready = True
        st.success(f"Detected emotion: {emotion}")

# Step 2: Chat UI
if st.session_state.chat_ready:
    st.subheader("💬 Chat with Assistant")

    # Display chat history
    for msg in st.session_state.messages:
        if msg["role"] == "system":
            continue
        sender = "🧑 You" if msg["role"] == "user" else "🤖 Assistant"
        st.markdown(f"**{sender}:** {msg['content']}")

    # Chat input (always visible)
    user_input = st.chat_input("Say something...")
    if user_input:
        st.session_state.messages.append({"role": "user", "content": user_input})

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=st.session_state.messages
        )
        assistant_reply = response.choices[0].message["content"]
        st.session_state.messages.append({"role": "assistant", "content": assistant_reply})

        # Save to conversation history (latest state)
        st.session_state.conversation_history.append({
            "emotion": st.session_state.current_emotion,
            "audio_file": st.session_state.audio_file_name,
            "chat": st.session_state.messages.copy()
        })

        st.rerun()
